In [3]:
from pathlib import Path
out_path_parquet = "/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04/part-000.parquet"
df = con.execute(""" Select * from read_parquet(?) """,[str(out_path_parquet)]).df()
df.head()

IOException: IO Error: No files found that match the pattern "/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04/part-000.parquet"

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
from dis import disco
import os
import glob
from pathlib import Path
path = Path(r"/home/faacosta0245695/conflit/conflit_warehouse/data/bronze")
def discovery (path_warehouse: Path, chunk_size=8192):
    for root, dirs, files in os.walk(path_warehouse):
        for file in files:
            if file.endswith('.parquet'):
                print(root, file)
discovery(path)

/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04 [] part-000.parquet


In [ ]:
import os
import hashlib
from pathlib import Path
from datetime import datetime
def compute_file_hash(path: Path, chunk_size=65536) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while chunk := f.read(chunk_size):
            h.update(chunk)
    return h.hexdigest()

def discovery(path_warehouse: Path, chunk_size=65536):
    records = []
    for root, _, files in os.walk(path_warehouse):
        for file in files:
            if file.endswith(".parquet"):
                file_path = Path(root) / file
                records.append({
                    "file_path": str(file_path),
                    "content_hash": compute_file_hash(file_path, chunk_size),
                    "file_size": file_path.stat().st_size,
                    "discovered_at": datetime.now(),
                    "source_partition": next((p for p in file_path.parts if p.startswith("date=")), None)
                })
    return records

# Example usage
bronze_path = Path("/home/faacosta0245695/conflit/conflit_warehouse/data/bronze")
files = discovery(bronze_path)
print(files[:2])  # peek first two

[{'file_path': '/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-04/part-000.parquet', 'content_hash': '6ae7cf49a930577cc4a3f216f2e09539fe1b9e7eb7ac0731b820ebc8189acd8a', 'file_size': 31338, 'discovered_at': datetime.datetime(2025, 9, 8, 15, 54, 59, 285006), 'source_partition': 'date=2025-09-04'}]


In [ ]:
import duckdb
def supports_merge(con: duckdb.DuckDBPyConnection) -> bool:
    try:
        con.execute("CREATE TEMP TABLE _a(x INT);")
        con.execute("CREATE TEMP TABLE _b(x INT);")
        con.execute("MERGE INTO _a a USING _b b ON a.x = b.x "
                    "WHEN MATCHED THEN UPDATE SET x=b.x "
                    "WHEN NOT MATCHED THEN INSERT VALUES (b.x);")
        return True
    except duckdb.Error:
        return False
    finally:
        con.execute("DROP TABLE IF EXISTS _a;")
        con.execute("DROP TABLE IF EXISTS _b;")
con = duckdb.connect()
supports_merge(con)

False

In [ ]:
import duckdb 
con = duckdb.connect("/home/faacosta0245695/conflit/conflit_warehouse/warehouse/database.db")
con.execute("""TRUNCATE ingest_manifest""").df()

,Count
0,1


In [1]:
import duckdb 
con = duckdb.connect("/home/faacosta0245695/conflit/conflit_warehouse/warehouse/database.db")
con.execute("""Select * from gold.refugee_stack_yearly""").df()


,origin_country_id,dest_country_id,country_origin,country_destination,refugees,asylum_seekers,idps,stateless,others_of_concern,host_community,cross_border_total,internal_total,population_of_concern,year,_ingest_run_id,_transform_version,_built_at
0,1,2,AFG,ALB,0,0,0,0,5,0,0,0,5,2020,20251002T140829,1,2025-10-02 16:08:30.022446
1,1,63,AFG,EGY,34,44,0,0,0,0,78,0,78,2020,20251002T140829,1,2025-10-02 16:08:30.022446
2,1,8,AFG,ARG,12,0,0,0,0,0,12,0,12,2020,20251002T140829,1,2025-10-02 16:08:30.022446
3,1,9,AFG,ARM,5,0,0,0,0,0,5,0,5,2020,20251002T140829,1,2025-10-02 16:08:30.022446
4,1,11,AFG,AUS,10659,1761,0,0,0,0,12420,0,12420,2020,20251002T140829,1,2025-10-02 16:08:30.022446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,220,192,ZWE,SWE,18,10,0,0,0,0,28,0,28,2020,20251002T140829,1,2025-10-02 16:08:30.022446
5484,220,193,ZWE,CHE,12,5,0,0,0,0,17,0,17,2020,20251002T140829,1,2025-10-02 16:08:30.022446
5485,220,196,ZWE,THA,9,0,0,0,0,0,9,0,9,2020,20251002T140829,1,2025-10-02 16:08:30.022446
5486,220,211,ZWE,USA,756,1041,0,0,0,0,1797,0,1797,2020,20251002T140829,1,2025-10-02 16:08:30.022446


In [ ]:
import duckdb 
con = duckdb.connect("/home/faacosta0245695/conflit/conflit_warehouse/warehouse/database.db")
def planner(con):
    dirty = con.execute(""" SELECT entity, partition_key
                                FROM dirty_partitions
                                WHERE status = 'dirty'
                                ORDER BY last_seen_at, entity, partition_key; """).df()
    for entity, partition_key in zip(dirty['entity'],dirty['partition_key']):
        plan_df  = con.execute(f""" 
            WITH candidates AS (
                SELECT mpl.file_path, mpl.content_hash, m.file_size, m.discovered_at, m.status
                FROM manifest_partition_link mpl
                JOIN ingest_manifest m
                    ON m.file_path = mpl.file_path
                AND m.content_hash = mpl.content_hash
                WHERE mpl.entity = ?
                    AND mpl.partition_key = ?
                ),
                latest_per_path AS (
                SELECT * EXCLUDE rn
                FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY file_path ORDER BY discovered_at DESC) rn
                    FROM candidates
                    WHERE status IN ('pending','processed')
                ) WHERE rn = 1
                )
                SELECT * FROM latest_per_path
                ORDER BY discovered_at ASC, file_path ASC;
    
            """,[entity, partition_key]).df()
        if plan_df.empty:
            pass

        yield {
            "entity": entity,
            "partition_key": partition_key,
            "inputs": plan_df.to_dict("records"),
        }        
for i in planner(con):
    print(i)

{'entity': 'refugee_displacement_conformed', 'partition_key': '2020', 'inputs': [{'file_path': '/home/faacosta0245695/conflit/conflit_warehouse/data/bronze/unhcr/date=2025-09-09/2020-2020-part-000.parquet', 'content_hash': '6ae7cf49a930577cc4a3f216f2e09539fe1b9e7eb7ac0731b820ebc8189acd8a', 'file_size': 31338, 'discovered_at': Timestamp('2025-09-09 11:55:06.614298'), 'status': 'pending'}]}


In [ ]:
import pandas as pd
import yaml
from pathlib import Path

def load_schema(path: Path):
    with open(path, "r") as f:
        schema = yaml.safe_load(f)
    return schema
def harmonize(df_raw: pd.DataFrame, schema: dict) -> pd.DataFrame:
    rename_map = {}
    casts = {}
    for col in schema["columns"]:
        canonical = col["name"]
        found = next((c for c in col["source"] if c in df_raw.columns), None)
        if found:
            rename_map[found] = canonical
        casts[canonical] = col["type"]
    
    df = df_raw.rename(columns=rename_map)
    
    for col in [c["name"] for c in schema["columns"]]:
        if col not in df.columns:
            df[col] = None
    
    for col, typ in casts.items():
        if typ == "BIGINT":
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        elif typ == "DOUBLE":
            df[col] = pd.to_numeric(df[col], errors="coerce")
        elif typ == "STRING":
            df[col] = df[col].astype("string")
        elif typ == "DATE":
            df[col] = pd.to_datetime(df[col], errors="coerce")
    
    for col in schema.get("lineage_columns", []):
        if col not in df.columns:
            df[col] = None
    
    df = df[[c["name"] for c in schema["columns"]] + schema.get("lineage_columns", [])]
    return df
raw = pd.read_csv("/home/faacosta0245695/conflit/conflit_warehouse/data/raw/unhcr/persons_of_concern.csv")
schema = load_schema("/home/faacosta0245695/conflit/conflit_warehouse/schemas/refugees_stack.yaml")

print(harmonize(raw,schema))

      country_origin country_destination  refugees  asylum_seekers     idps  \
0                AFG                 AFG         0               0  2553390   
1                AFG                 EGY        28              36        0   
2                AFG                 ARG        12               0        0   
3                AFG                 ARM         5               0        0   
4                AFG                 AUS     11585            1710        0   
...              ...                 ...       ...             ...      ...   
34706            ZWE                 SWE        24               0        0   
34707            ZWE                 CHE         9               5        0   
34708            ZWE                 USA       676            3037        0   
34709            ZWE                 ZMB        10              15        0   
34710            ZWE                 ZWE         0               0        0   

       stateless  hst     ooc  year _bronze_file _i

In [1]:
import duckdb 
con = duckdb.connect("/home/faacosta0245695/conflit/conflit_warehouse/warehouse/database.db")
con.execute(""" Select * from silver.refugee_displacement_conformed """).df()

,origin_country_id,dest_country_id,country_origin,country_destination,refugees,asylum_seekers,idps,stateless,host_community,others_of_concern,year,origin_region_unhcr,origin_sub_region_unhcr,dest_region_unhcr,dest_sub_region_unhcr,_build_at,_ingest_run_id,_transform_version,row_hash
0,1,2,AFG,ALB,0,0,0,0,0,5,2020,Asia and the Pacific,Southern Asia,Europe,Southern Europe,2025-10-02 11:27:44.359891+02,20251002T092744,1,0c7461fb8540bb26bfa983ed50b140f061f87c4d698157...
1,1,63,AFG,EGY,34,44,0,0,0,0,2020,Asia and the Pacific,Southern Asia,Middle East and North Africa,Northern Africa,2025-10-02 11:27:44.359891+02,20251002T092744,1,14114346ac6203d678a3fbad17e67bd9e1b0dab2010de4...
2,1,8,AFG,ARG,12,0,0,0,0,0,2020,Asia and the Pacific,Southern Asia,Americas,Latin America and the Caribbean,2025-10-02 11:27:44.359891+02,20251002T092744,1,d1ba929c7e5c11afd717b693f5a65dfb1c9fb30663a82c...
3,1,9,AFG,ARM,5,0,0,0,0,0,2020,Asia and the Pacific,Southern Asia,Europe,Western Asia,2025-10-02 11:27:44.359891+02,20251002T092744,1,7eaa1d771c34ae42019f7404b7705a7361f12307d95cc0...
4,1,11,AFG,AUS,10659,1761,0,0,0,0,2020,Asia and the Pacific,Southern Asia,Asia and the Pacific,Australia and New Zealand,2025-10-02 11:27:44.359891+02,20251002T092744,1,bf5aa3118795889c22bca3df4db62a69b5bcc84924e239...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,220,192,ZWE,SWE,18,10,0,0,0,0,2020,Southern Africa,Sub-Saharan Africa,Europe,Northern Europe,2025-10-02 11:27:44.359891+02,20251002T092744,1,25f2f00c70898042d83aeb8f1a575bc86d8dea27bfb61a...
5484,220,193,ZWE,CHE,12,5,0,0,0,0,2020,Southern Africa,Sub-Saharan Africa,Europe,Western Europe,2025-10-02 11:27:44.359891+02,20251002T092744,1,d76c901586e8f89a30ef3b3190bec58e8856a840811e71...
5485,220,196,ZWE,THA,9,0,0,0,0,0,2020,Southern Africa,Sub-Saharan Africa,Asia and the Pacific,South-eastern Asia,2025-10-02 11:27:44.359891+02,20251002T092744,1,0a8927eb70372b48dee422b6b401abb1a6347d4b410d9e...
5486,220,211,ZWE,USA,756,1041,0,0,0,0,2020,Southern Africa,Sub-Saharan Africa,Americas,Northern America,2025-10-02 11:27:44.359891+02,20251002T092744,1,c70b6b86a6ea7c773809f623ee3be22301af294136c974...
